The project is based on NARCIS (National Academic Research and Collaboration Information System) harvested meta-data from different PhD thesis repositories in The Netherlands (NARCIS, n.d.). NARCIS aggregates the information on scholarly work across Dutch academic institutions. NARCIS followed standardized protocols during the extraction, ensuring consistency and interoperability across various data sources.

By now the NARCIS dataset has been included into OpenAIRE.

To obtain the original dataset, please contact @tamarinde [on Github](https://github.com/tamarinde).

This script is intended to clean the original dataset that was obtained from the XXX study. The raw dataset should be placed in

`data/raw/pairs_sups_phds.csv`

Make sure that the columns have the following names: `thesis_identifier`,`contributor`,`contributor_order`,`institution`,`author_name`,`title`,`year`,`language`

This notebook cleans the raw data in the following ways

1. Remove wrongly detected conributor entries that are not people. For this, we make use of spacy and the `xx_ent_wiki_sm` model
2. Standardize the contributor names to "Last name, Initials" -> let's check here if we are actually throwing away information here and if we should keep the first names
3. Aggregate all rows per publication/PhD student into one row with all the contributors
4. Pivot the dataset, to get to one row per publication/PhD student, with one columns per contributor
5. Export the dataset to `data/cleaned/sampled_pubs.csv`

## Clean and restructure dataset

In [ ]:
# Import dependencies
import pandas as pd

# custom functions
from src.clean_names_helpers import remove_non_person_contributors_and_export
from src.clean_names_helpers import format_name_to_lastname_initials
from src.clean_names_helpers import ensure_and_load_spacy_model

In [ ]:
# Load, and if notvailable, download the spacy nlp model 
model_name = "xx_ent_wiki_sm" # multilingual NER model
nlp = ensure_and_load_spacy_model(model_name)

In [ ]:
# Initialize values
NROWS = 100 # None for all

# names that spacy does not recognize as such
# NOTE: Add the verbatim names here, not the standardized target notation 
# This list can be fed from removed_contributors.csv that is created when running the script
WHITELIST = [ 
    "Oosterlaan, J.",
    "Nollet, F."
    ] 

# non-people's names that don't get filtered out by spaCy 
BLACKLIST = [
    "Cardiology"
]

removed_contributors = []

In [ ]:
# Read data
pairs_raw = pd.read_csv("data/raw/pairs_sups_phds.csv", nrows=NROWS)
pairs_raw = pairs_raw.convert_dtypes() # make sure all integer columns are integer dtype 
pairs_raw.head()

In [ ]:
# remove duplicates
pairs_filtered = pairs_raw.drop_duplicates() 

# Remove rows where 'contributor' is NA
pairs_filtered = pairs_filtered.dropna(subset=['contributor'])

# remove contributors that aren't people
csv_path = "data/removed_contributors.csv"
pairs_filtered = remove_non_person_contributors_and_export(pairs_filtered, csv_path, nlp, WHITELIST, BLACKLIST)

print(f"{len(pairs_filtered)} columns are left.")

In [ ]:
# Standardize names
pairs_std = pairs_filtered
# Apply name standardization to the contributor column
pairs_std['contributor'] = pairs_filtered['contributor'].apply(format_name_to_lastname_initials)

In [ ]:
# Group by publication
aggregated = pairs_std.groupby([
        'integer_id', 
        'thesis_identifier', 
        'institution', 
        'author_name', 
        'title', 
        'year', 
        'language'
    ])
        
# Aggregate contributors into a list
aggregated = aggregated.agg(list)

aggregated = aggregated.reset_index()
    
# make sure the contributor is a sequence from 1 to n_contributors
aggregated['contributor_order'] = aggregated['contributor_order'].apply(lambda lst: list(range(1, len(lst) + 1)))


In [ ]:
# Pivot the dataset, to get to one row per dissertation, with the contributors in columns

# Initialize a list to hold publication data dictionaries
pubs_list = []

# Iterate over each aggregated group
for _, row in aggregated.iterrows():
    # Initialize a dictionary with publication information
    pub_dict = {col: row[col] for col in ['integer_id', 'thesis_identifier', 'institution', 'author_name', 'title', 'year', 'language']}
    
    # Get the list of contributors and their orders for this publication
    contributors = row['contributor']
    contributor_orders = row['contributor_order']
    
    # Add contributors to the dictionary using dynamic keys
    for order in sorted(set(contributor_orders)):  # Ensure unique and sorted order numbers
        if order - 1 < len(contributors):  # Check to prevent index error
            pub_dict[f'contributor_{order}'] = contributors[order - 1]
    
    # Append the publication dictionary to the list
    pubs_list.append(pub_dict)

# Convert the list of dictionaries to a DataFrame
pubs = pd.DataFrame(pubs_list)

# Ensure correct data types and fill missing values with a suitable placeholder if necessary
pubs = pubs.convert_dtypes()

len(pubs)

In [ ]:
# Export a sample of the dataset to easily share it
seed = 42 # fixed seed

# Sample 100 random lines from the DataFrame
#sampled_pubs = pubs.sample(n=50, random_state=seed)

# Export the sampled DataFrame to a CSV file
#sampled_pubs.to_csv('data/cleaned/sampled_pubs.csv', index=False)

## Call Scopus API

In [ ]:
import src.api_helpers as api_helpers

from src.api_helpers import find_publications
from src.api_helpers import find_first_publication
from src.api_helpers import common_pub_author_and_contributor_1_row

import importlib
importlib.reload(api_helpers)

# to find the configuration file, run
# import pybliometrics
# pybliometrics.scopus.utils.constants.CONFIG_FILE

In [ ]:
# Test if I have the connection, in case my authentication does not work
from pybliometrics.scopus import ScopusSearch
s = ScopusSearch('ISSN(1532-849X) AND PUBYEAR IS 2010',subscriber=False)
print(s.results)

In [ ]:
# look up publications of one contributor
author_last_name = "van Neerven"
author_initials = "J"
publications = find_publications(author_last_name, author_initials)

publications.head()

In [ ]:
pubs_sample = pubs.head(1)
pubs_sample

In [ ]:
# go through a test set of the publications

#common_pub_author_and_contributor_1_row(pubs_sample)

author = pubs_sample['author_name'].iloc[0]
contributor = pubs_sample['contributor_1'].iloc[0]

publications_author = find_publications(*author.split(', ')) if author is not None else []
publications_contributor = find_publications(*contributor.split(', ')) if contributor is not None else []

# # Look up both author_name and contributor_1 and find the first common publication, if there is any
# pubs_sample['common_publication'] = pubs_sample.apply(common_pub_author_and_contributor_1_row, axis=1)
# pubs_sample

contributor

So far, so good. The next step would be to get the first author each that the API returns and sees if they both share a publication. If that's not the case, the code should move to the next match for author_name, then contributor_1 and so on. The added layer of iteration is somewhat looking for plausible matches, namely matches that share a publication. We might still have a lot of false positives, this way though.

Another layer we can build in is to verify the affilliation of the authors (that is in the matching publication). We can then whitelist papers where the PhD name is listed under the correct affiliation on that paper. As a stand-in we can also only count contributor_1 that had the same affiliation. To be determined if this works.

We can also get some diagnostics on how many name matches we got per name in the dataframe. This should give us some idea on how many API calls we need to do and how certain we are with the name matching. 
We should also check how much of TiU's call budget we would us up to go throught the entire list. 

There seems to be an affiliation ID